# 光子の干渉データ解析

解析に入る前に`python`プログラミング言語と数値計算等に用いられるパッケージについて説明していきます。

# python 基礎

## 変数
変数の定義。  
整数型、不動小数点型、文字列型など一般的なデータ型が準備されています。  
ユーザーはとくにデータ型を明示することなく変数を宣言することができます。

In [ ]:
a = 1 # 整数型
x = 3.14 # 不動小数点型
s = 'hello' # 文字列型

変数を表示するには`print()`関数を使います。

In [ ]:
print(s)

もしも`C/C++`などで同じ変数宣言を行おうとすると以下のようになります。  
データ型を明示するために倍程度の文字数を必要としていることがわかります。  
人間がタイプできるスピードが同じだとすると、`python`だと実質的なコードをより多く書くことができるわけです。

In [ ]:
int a = 1;
double x = 3.14;
std::string x = 'hello';

変数同士の四則演算が可能です。

In [ ]:
a1 = 2
a2 = 3
print(a1+a2) # 足し算
print(a1-a2) # 引き算
print(a1*a2) # 掛け算
print(a1/a2) # 割り算

文字列同士の演算はどうでしょうか？

In [ ]:
s1 = 'hello'
s2 = 'world'
print(s1+s2)

# リスト,配列
リスト、配列は変数の組み(数学的に言えばベクトル)です。  
プログラミングでは「リストの各要素について処理を行う」というような操作がよく行われます。

In [ ]:
arr = [1,2,3,4,5]

配列同士の演算を考えてみます。

In [ ]:
a1 = [1,2,3]
a2 = [4,5,6]
print(a1+a2)

# ループ

In [ ]:
for e in arr:
  print(e)
  

# 条件分岐

In [ ]:
a = 1
if a == 1:
  print('a is one')
else:
  print('a is not one')

In [ ]:
for e in arr:
  if e > 3:
    print(e)

# 関数

In [ ]:
def fun(x):
  return x**2

In [ ]:
for e in arr:
  print(fun(e))

# numpy

In [ ]:
import numpy as np

# numpy配列

In [ ]:
x1 = np.array([1,2,3,4,5])
x2 = np.array([6,7,8,9,10])
y = x1**2
y.std()

数値計算`numpy`、データ解析`pandas`パッケージの読み込み。  
`C++/C`の経験者であれば、`import`は`#include`に対応すると理解してください。

# pandas

In [ ]:
import pandas as pd

### データファイル (excel) の読み込み
`pandas.read_excel()`関数はエクセルファイルを`pandas`のデータフレームというコンテナ(以下では`df`としている)にロードする。  
以下ではコンテナ`df`にアクセス、処理することで表計算処理が可能となる。  
`photo_if_data.xlsx`は過去の実験データ。  
データフォーマットは　
- 1列目 光電子増倍管の位置 (x)
- ２列目 レーザー強度 (V)
- 3列目 カウント数 (N)
- 4列目 N/V
としている。  
各自、ファイルにおけるデータ列の並びを考慮して以下のコードを書き換える。  
なお、引数で用いられている`sheet_name`はエクセルのシートの名前, `usecols`は読み込む列番号 (ここでは0,1,2,3としている),  
`names`で各列の名前を定義している。  
他にも`read_excel()`関数には様々なオプションが用意されている。  

<div class="alert alert-block alert-info">
<b>参考 csvファイルの読み込み</b><br>
pandas.read_csv() 関数はcsvファイルをpandasのデータフレームというコンテナ(以下ではdfとしている)にロードする。<br>
以下dfにアクセスを処理することで表計算処理が可能となる。 <br>
例として、データフォーマットは一列目光電子増倍管の位置、２列目　光電子増倍管のカウント数/レーザー強度。<br>
photo_if_data.csvは過去の実験データ。  
</div>


In [ ]:
df = pd.read_excel('photo_if_data.xlsx', sheet_name='Sheet2',usecols=[0,1,2,3],names=('X', 'N','V','NV'))
df = df.dropna()
# df = pd.read_csv('https://github.com/MakotoUchida/b3exp-photo-if/raw/main/photo_if_data.csv',names=('X','NV'))

最初の５行を見てみる。

In [ ]:
df.tail()

In [ ]:
df.NV

最後の５行を見るには？

### 描画パッケージmatplotlibの読み込み
パッケージ名が長いので`plt`と名前をつけるのが通例。

In [ ]:
import matplotlib.pyplot as plt

### グラフ作成

In [ ]:
plt.plot(df.X,df.NV,'o-')
plt.xlabel('X [mm]')


In [ ]:
df['V'].hist(bins=50)

### 理論カーブの考察

レーザースポットの広がりを考慮すると、ガウス分布関数
$$
G(x) = Ae^{-\frac{1}{2}\frac{(x-m)^2}{s^2}}
$$
干渉パターン
$$
I(x) = (1+cos(ksin\theta x)) = (1+cos((2\pi/d)x)
$$
実験データはおそらくこの２つの関数の掛け算で表現できると予想される。  
実際に２つの関数をかけ合わせたグラフを作成する。

In [ ]:
x = np.linspace(0,10,100)
y = 3000*np.exp(-0.5*(x-5)**2/2**2)*(1+np.cos(((2*np.pi*(x+0.4))/1.2)))
# y = 50000*np.exp(-0.5*(x-4)**2/2**2)*(1+np.cos(((2*np.pi*(x+0.43))/1)))

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
_ = ax.plot(df.X,df.NV,'o-')
_ = ax.plot(x,y,'-')

### フィッティング関数の読み込み
上記のように、おおよそのパラーメータの値を推測してデータを再現してもよいが  
正しくは最適なパラメータをフィッティングで求めることとなる。  
フィッティングには`scipy.optimize.curve_fit()`関数を用いることができる。

In [ ]:
from scipy.optimize import curve_fit

使い方は、簡単に言うと
```python
curve_fit(関数, データx, データy, 初期値)
```
とすればよい。  
返り値として最適化されたパラメータのリスト （+covariant matrix）。


In [ ]:
help(curve_fit)

### フィッティング関数の定義
`fitFun()`の`a,b,c,d,e`がパラメータ。

In [ ]:
def fitFun(x,a,b,c,d,e):
    return a*np.exp(-0.5*(x-b)**2/c**2)*(1+np.cos((2*np.pi*(x+e)/d)))

In [ ]:
par,cov = curve_fit(fitFun,df.X,df.NV,p0=(3000,5,2,1,0.5))

In [ ]:
par

結果を表示してみる。

In [ ]:
fig = plt.figure(figsize=(6,4))
y = fitFun(x,par[0],par[1],par[2],par[3],par[4])
ax = fig.add_subplot(111)
_ = ax.plot(df.X,df.NV,'o-',label='data')
_ = ax.plot(x,y,'-',label='fit')
plt.legend(loc='upper left')
plt.savefig('pif.png') # グラフをpngファイルとして保存する関数。

In [ ]:
y = fitFun(x,par[0],par[1],par[2],par[3],par[4])

In [ ]:
y = fitFun(x,par[0],par[1],par[2],par[3],par[4])

$$
\chi^2 = \sum_{i=1}^n (y_i - f(x_i))^2
$$

$$
\frac{\partial \chi^2}{\partial a} =0,\frac{\partial \chi^2}{\partial b} =0,...
$$